In [1]:
import numpy as np
import csv
import nltk
import itertools
from datetime import datetime
import rnn_with_numpy as rnn
reload(rnn)

<module 'rnn_with_numpy' from 'rnn_with_numpy.pyc'>

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/anderson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
SENTENCE_START = '_SETENCE_START'
SENTENCE_END = '_SENTENCE_END'
UNKNOWN_TOKEN = '_UNKNOWN_TOKEN'
VOCAB_SIZE = 8000

In [10]:
t = datetime.now()
with open('/home/anderson/Dropbox/reddit_comments_small.csv', 'rt') as f:
    reader = csv.reader(f, skipinitialspace=True)
    
    # Split full comments into sentences
    sentences = itertools.chain(
        *[nltk.sent_tokenize(x[0].decode('UTF-8').lower()) for x in reader])
    print 'Split Done'
    
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (SENTENCE_START, x, SENTENCE_END)
                    for x in sentences ]
    print 'Finished: ', (datetime.now() - t).total_seconds()

start reading comments! (wait for minutes...)
Split Done
Finished:  0.653252


In [5]:
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(st) for st in sentences ]

In [6]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "%d unique words tokens." % len(word_freq.items())

17197 unique words tokens.


### Word Indexing

아래 2개의 함수를 사용해서 word -> index 또는 그 반대로 변환을 해 줄수 있습니다.

* index_to_word(index) : list -> word
* word_to_index(word) : dict -> integer

In [7]:
most_common_freq = word_freq.most_common(VOCAB_SIZE-1)
index_to_word = [x[0] for x in most_common_freq]
index_to_word.append(UNKNOWN_TOKEN)
word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])

print "VOCAB_SIZE: %d" % VOCAB_SIZE
print "Unknown token Index:", word_to_index[UNKNOWN_TOKEN]
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % \
          (most_common_freq[-1][0], most_common_freq[-1][1])

VOCAB_SIZE: 8000
Unknown token Index: 7999
The least frequent word in our vocabulary is '//imgur.com/idqgygh' and appeared 1 times.


### Make Unknown Tokens

링크, 이상한 단어들.. 등등. 모든 단어들을 모두 외우고 있을수는 없습니다.<br>
따라서 가장 많이 나온 단어들 (word_freq.most_common(VOCAB_SIZE-1)) 을 제외하고, 그 외 단어들은 UNKNOWN_TOKEN으로 대체합니다.

In [8]:
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else UNKNOWN_TOKEN for w in sent]

### Train Data

In [9]:
# Create the training data
x_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

print x_train[1]
print y_train[1]

[0, 68, 522, 67, 35, 91, 20, 7999, 32, 12, 3026, 34]
[68, 522, 67, 35, 91, 20, 7999, 32, 12, 3026, 34, 1]


### Model

In [10]:
reload(rnn)
model = rnn.RNNNumpy(VOCAB_SIZE)
o, s = model.forward_propagation(x_train[9])
p = model.predict(x_train[9])

In [11]:
print 'error cost:', model.cross_entropy(x_train[26], y_train[26])
model.bptt(x_train[26], y_train[26])
model.calculate_gradients(x_train[26], y_train[26])

error cost: 8.98747007115


### Train!

In [12]:
model.train(x_train, y_train, npoch=len(x_train))

Start Training
Total Data:  11587
2016-09-14 21:11:37.062952: i=0, cost=8.98722103854
2016-09-14 21:11:44.719646: i=100, cost=8.98706676536
2016-09-14 21:11:52.438135: i=200, cost=8.98683016155
2016-09-14 21:12:00.514727: i=300, cost=8.98473871271
2016-09-14 21:12:08.149691: i=400, cost=8.98729998738
2016-09-14 21:12:15.803158: i=500, cost=8.97605857328
2016-09-14 21:12:22.979648: i=600, cost=8.97772101904
2016-09-14 21:12:30.504268: i=700, cost=8.97798193959
2016-09-14 21:12:37.881149: i=800, cost=8.9614174036
2016-09-14 21:12:45.391671: i=900, cost=8.94699520533
2016-09-14 21:12:53.085263: i=1000, cost=8.83650153805
2016-09-14 21:13:00.471641: i=1100, cost=8.51452153067
2016-09-14 21:13:07.530092: i=1200, cost=8.85190189799
2016-09-14 21:13:15.423381: i=1300, cost=8.86469610436
2016-09-14 21:13:22.147843: i=1400, cost=6.01450408332
2016-09-14 21:13:30.704227: i=1500, cost=6.74473788643
2016-09-14 21:13:39.215563: i=1600, cost=8.63738353483
2016-09-14 21:13:46.629712: i=1700, cost=9.5

In [59]:
def convert_idx_to_sentence(index):
    return ' '.join([index_to_word[i] for i in index])

def convert_sentence_to_idx(sentence):
    return  [ word_to_index[w] for w in nltk.word_tokenize(sentence)]

In [68]:
test1 = convert_sentence_to_idx(SENTENCE_START+ " in ny everything is so")
result1 = model.predict(test1)
print 'result1:', result1
print convert_idx_to_sentence(test1)
print convert_idx_to_sentence(result1)

print 

result1: [   7    3 7999    3    3 7999]
_SETENCE_START in ny everything is so
i the _UNKNOWN_TOKEN the the _UNKNOWN_TOKEN

